In [2]:
import pandas as pd
import yfinance as yf # 주가 데이터 받기위한 라이브러리
import quantstats as qs # 리포트 형식으로 시각화해주는 라이브러리

In [3]:
# yahoo finance에서 필요 종목들의 공통 시작일부터 종가 데이터 받기
def get_yahoo_data(tickers, get_type="Adj Close"):
    df = yf.download(tickers)
    df = df[get_type]
    df.dropna(inplace=True)
    return df


In [5]:
# 리밸런싱 하는 날의 데이터만 뽑기(월말 데이터만 추출)  
def get_rebal_date(df, rebal="month"):
    res_df = pd.DataFrame()
    df["year"] = df.index.year
    df["month"] = df.index.month
    df["day"] = df.index.day
    days_df = df.groupby(["year","month"])["day"].max()
    for i in range(len(days_df)):
        if days_df.iloc[i] > 25:
            day = "{}-{}-{}".format(days_df.index[i][0], days_df.index[i][1], days_df.iloc[i])
            res_df = pd.concat([res_df,df[df.index==day]])
    return res_df


# 필요한 종목의 ticker들
tickers_canary = ["SPY","EEM","EFA","AGG"]
tickers_g4 = ["QQQ","EEM","EFA","AGG"]
tickers_g12 = ["SPY", "QQQ", "IWM","VGK","EWJ", "EEM", "VNQ","DBC","GLD", "TLT", "HYG", "LQD"]
tickers_safe = ["TIP","DBC","BIL","IEF","TLT","LQD","AGG"]
tickers_all = list(set(tickers_canary+tickers_g4+tickers_g12+tickers_safe))

In [6]:
tickers_all

['TLT',
 'LQD',
 'IWM',
 'VNQ',
 'EEM',
 'AGG',
 'GLD',
 'BIL',
 'SPY',
 'TIP',
 'EWJ',
 'DBC',
 'VGK',
 'IEF',
 'HYG',
 'EFA',
 'QQQ']

In [7]:
data = get_yahoo_data(tickers_all) # 모든 데이터
rebal_data = get_rebal_date(data) # 월말 데이터
rebal_data.head(3)

[*********************100%***********************]  17 of 17 completed


,AGG,BIL,DBC,EEM,EFA,EWJ,GLD,HYG,IEF,IWM,LQD,QQQ,SPY,TIP,TLT,VGK,VNQ,year,month,day
Date,,,,,,,,,,,,,,,,,,,,
2007-05-31,63.198601,83.139603,23.811016,31.132423,50.877041,45.920582,65.540001,39.796047,57.305073,68.209305,58.531075,41.566051,113.102249,63.369164,55.141476,42.191029,41.196838,2007,5,31
2007-06-29,62.960529,83.502655,23.988365,32.277569,50.713799,45.637508,64.269997,38.541496,57.037048,67.228279,58.056953,41.765251,111.448639,63.233059,54.582661,42.042316,37.485935,2007,6,29
2007-07-31,63.636211,83.836311,24.445730,32.508060,49.552219,45.385895,65.790001,37.046040,58.324272,62.524731,57.578491,41.703846,107.959122,64.687935,56.394283,40.847084,34.389839,2007,7,31


In [8]:
rebal_data.tail(3)

,AGG,BIL,DBC,EEM,EFA,EWJ,GLD,HYG,IEF,IWM,LQD,QQQ,SPY,TIP,TLT,VGK,VNQ,year,month,day
Date,,,,,,,,,,,,,,,,,,,,
2022-08-31,100.057610,91.049080,25.719999,39.430000,61.700001,53.590000,159.270004,73.517952,100.348473,182.555679,108.276451,298.733154,393.563293,112.472000,111.104614,51.098854,92.013313,2022,8,31
2022-09-30,95.910202,91.237328,23.910000,34.880001,56.009998,48.849998,154.669998,70.768120,95.597000,164.919998,101.801689,267.260010,357.179993,104.900002,101.955032,46.150002,80.169998,2022,9,30
2022-10-31,94.684006,91.381996,25.120001,34.189999,59.310001,49.990002,151.910004,73.150002,94.207008,183.330002,101.042999,277.950012,386.209991,106.389999,95.874001,50.040001,82.980003,2022,10,31


In [9]:
canary_data = rebal_data[tickers_canary]
profit = rebal_data.pct_change()
n=12
for i in range(n, rebal_data.shape[0]):
    print(rebal_data.index[i])
    
    # 1-3-6-12 점수 구하기
    m1 = (canary_data.iloc[i]-canary_data.iloc[i-1])/canary_data.iloc[i-1]
    m3 = (canary_data.iloc[i]-canary_data.iloc[i-3])/canary_data.iloc[i-3]
    m6 = (canary_data.iloc[i]-canary_data.iloc[i-6])/canary_data.iloc[i-6]
    m12 = (canary_data.iloc[i]-canary_data.iloc[i-12])/canary_data.iloc[i-12]
    score = m1*12 + m3*4 + m6*2 +m12

    buy = dict()
    if min(score) <= 0: # 안전자산
        safe = rebal_data[tickers_safe].iloc[i] / rebal_data[tickers_safe].iloc[i-n:i].mean()
        safe_top3 = safe.nlargest(3)
        for j in range(3):
            try:
                if safe_top3[j] > safe["BIL"]:
                    name = safe_top3.index[j]
                else:
                    name = "BIL"
                buy[name] += (1/3)*100
            except:
                buy[name] = (1/3)*100
        print("safe : ", buy)
    else: # 공격형(g4) 또는 중도형(g12)
        g4 = rebal_data[tickers_g4].iloc[i] / rebal_data[tickers_g4].iloc[i-n:i].mean()
        g4_top1 = g4.nlargest(1)
        buy[g4_top1.index[0]] = 100
        print("G4 : ", buy)
        buy=dict()
        g12 = rebal_data[tickers_g12].iloc[i] / rebal_data[tickers_g12].iloc[i-n:i].mean()
        g12_top6 = g12.nlargest(6)
        for j in range(6):
            buy[g12_top6.index[j]] = (1/6)*100 
        print("G12 : ", buy)

2008-05-30 00:00:00
safe :  {'DBC': 33.33333333333333, 'TIP': 33.33333333333333, 'IEF': 33.33333333333333}
2008-06-30 00:00:00
safe :  {'DBC': 33.33333333333333, 'TIP': 33.33333333333333, 'IEF': 33.33333333333333}
2008-07-31 00:00:00
safe :  {'DBC': 33.33333333333333, 'TIP': 33.33333333333333, 'IEF': 33.33333333333333}
2008-08-29 00:00:00
safe :  {'DBC': 33.33333333333333, 'TIP': 33.33333333333333, 'TLT': 33.33333333333333}
2008-09-30 00:00:00
safe :  {'TLT': 33.33333333333333, 'IEF': 33.33333333333333, 'BIL': 33.33333333333333}
2008-10-31 00:00:00
safe :  {'TLT': 33.33333333333333, 'IEF': 33.33333333333333, 'BIL': 33.33333333333333}
2008-11-28 00:00:00
safe :  {'TLT': 33.33333333333333, 'IEF': 33.33333333333333, 'BIL': 33.33333333333333}
2008-12-31 00:00:00
safe :  {'TLT': 33.33333333333333, 'IEF': 33.33333333333333, 'AGG': 33.33333333333333}
2009-01-30 00:00:00
safe :  {'TLT': 33.33333333333333, 'IEF': 33.33333333333333, 'AGG': 33.33333333333333}
2009-02-27 00:00:00
safe :  {'TLT': 3

In [12]:
import pandas as pd
import yfinance as yf # 주가 데이터 받기위한 라이브러리
import quantstats as qs # 리포트 형식으로 시각화해주는 라이브러리

# yahoo finance에서 필요 종목들의 공통 시작일부터 종가 데이터 받기
def get_yahoo_data(tickers, type="Adj Close"):
  df = yf.download(tickers)
  # df = df["Close"]
  df = df[type]
  df.dropna(inplace=True)
  return df
  
def get_rebal_date(df, rebal="month"):
    res_df = pd.DataFrame()
    df["year"] = df.index.year
    df["month"] = df.index.month
    df["day"] = df.index.day
    days_df = df.groupby(["year","month"])["day"].max()
    for i in range(len(days_df)):
        if days_df.iloc[i] > 25:
            day = "{}-{}-{}".format(days_df.index[i][0], days_df.index[i][1], days_df.iloc[i])
            res_df = pd.concat([res_df,df[df.index==day]])
    return res_df
 
tickers_canary = ["SPY","EEM","EFA","AGG"]
tickers_g4 = ["QQQ","EEM","EFA","AGG"]
tickers_g12 = ["SPY", "QQQ", "IWM","VGK","EWJ", "EEM", "VNQ","DBC","GLD", "TLT", "HYG", "LQD"]
tickers_safe = ["TIP","DBC","BIL","IEF","TLT","LQD","AGG"]
tickers_all = list(set(tickers_canary+tickers_g4+tickers_g12+tickers_safe)

data = get_yahoo_data(tickers_all) # 모든 데이터
rebal_data = get_rebal_date(data) # 월말 데이터
rebal_data.head(3)

SyntaxError: '(' was never closed (3470796124.py, line 29)

In [13]:
# 공격형
baa_g4 = pd.DataFrame(columns=tickers_all)
res = [] # 총 자산 시작은 100

canary_data = rebal_data[tickers_canary]
profit = rebal_data.pct_change()
n=12
for i in range(n, rebal_data.shape[0]):
    # print(canary_data.index[i], profit.index[i])

    m1 = (canary_data.iloc[i]-canary_data.iloc[i-1])/canary_data.iloc[i-1]
    m3 = (canary_data.iloc[i]-canary_data.iloc[i-3])/canary_data.iloc[i-3]
    m6 = (canary_data.iloc[i]-canary_data.iloc[i-6])/canary_data.iloc[i-6]
    m12 = (canary_data.iloc[i]-canary_data.iloc[i-12])/canary_data.iloc[i-12]
    score = m1*12 + m3*4 + m6*2 +m12

    buy = dict()
    if min(score) <= 0:
        safe = rebal_data[tickers_safe].iloc[i] / rebal_data[tickers_safe].iloc[i-n:i].mean()
        safe_top3 = safe.nlargest(3)
        for j in range(3):
            try:
                if safe_top3[j] > safe["BIL"]:
                    name = safe_top3.index[j]
                else:
                    name = "BIL"
                buy[name] += (1/3)*100
            except:
                buy[name] = (1/3)*100
    else:
        aggresive = rebal_data[tickers_g4].iloc[i] / rebal_data[tickers_g4].iloc[i-n:i].mean()
        agg_top1 = aggresive.nlargest(1)
        buy[agg_top1.index[0]] = 100
    
    if i == n:
        one = pd.DataFrame([list(buy.values())], columns=list(buy.keys()), index = [rebal_data.index[i]])
        baa_g4 = pd.concat([baa_g4, one])
        res.append(100)
    else:
        total = sum(((1+profit.iloc[i])*baa_g4.iloc[-1]).fillna(0))
        res.append(total)
        one = pd.DataFrame([list(buy.values())], columns=list(buy.keys()), index = [rebal_data.index[i]]) * total / 100
        baa_g4 = pd.concat([baa_g4, one])

baa_g4["Total"] = res
baa_g4.tail()

,TLT,LQD,IWM,VNQ,EEM,AGG,GLD,BIL,SPY,TIP,EWJ,DBC,VGK,IEF,HYG,EFA,QQQ,Total
2022-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,346.260779,NaN,NaN,NaN,173.130390,NaN,NaN,NaN,NaN,NaN,519.391169
2022-07-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,344.047819,NaN,NaN,NaN,172.023910,NaN,NaN,NaN,NaN,NaN,516.071729
2022-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,342.779132,NaN,NaN,NaN,171.389566,NaN,NaN,NaN,NaN,NaN,514.168698
2022-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,502.816171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,502.816171
2022-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,335.742300,NaN,NaN,NaN,167.871150,NaN,NaN,NaN,NaN,NaN,503.613451


In [15]:
qs.reports.html(baa_g4["Total"])

/Users/minsu/miniforge3/envs/finance/lib/python3.10/site-packages/quantstats/stats.py:968: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  returns = returns.pivot('Year', 'Month', 'Returns').fillna(0)
